# Yosys inference of memories

This example demonstrates direct inference of a dual port (simplex) memory description into a FPGA hard memory block (here: Lattice ECP5 DP16KD primitive).
It requires a recent (>= v0.12) release of yosys using the improved memory mapping.

In [1]:
from myirl.emulation.myhdl import *
from myirl.test.test_array import r1w1, SigArray

## Simple read and write port memory

This implementation uses bypass logic when a write is occuring during a read from the same address.

In [2]:
@block
def transparent_r1w1(
        clk : ClkSignal,
        we  : Signal,
        addr_r: Signal,
        addr_w: Signal,
        din: Signal,
        dout: Signal.Output,
        ram : SigArray,
        MODE = False, # Conditional compilation flag w/o type annotation
        DWIDTH=16, AWIDTH=10
    ) -> IRL:

    @always(clk.posedge)
    def mem_rw():
        if we and addr_w == addr_r:
            dout.next = din  #Forward
        else:
            dout.next = ram[addr_r][DWIDTH:]

        if we:
            ram[addr_w].next = din
        
    return instances()


In [3]:
def memtest(MODE = 0, STYLE = 1, data_w=16, addr_w=6, mem = r1w1):
    c = ClkSignal(name = 'clk')
    c.init = True
    wren = Signal(bool(), name = 'we')
    ra, wa = [ Signal(intbv()[addr_w:]) for n in ['addr_write', 'addr_read'] ]
    a, q = [ Signal(intbv()[data_w:]) for n in ['a', 'q'] ]

    ram_data = SigArray([ intbv(v)[data_w:] for v in range(2 ** addr_w)],
        name='ram_sig', init=True)


    inst = mem(clk=c, we=wren, addr_r=ra, addr_w=wa, din=a, dout=q, MODE=False, ram = ram_data,
        AWIDTH=addr_w, DWIDTH=data_w)

    return inst


## Elaboration into hardware

In [4]:
from myirl.targets import pyosys

Elaborate memory unit and emit CXXRTL code:

In [5]:
dw = 16

tgt = pyosys.RTLIL("memtest%d" % dw)

MEM = transparent_r1w1
# MEM = r1w1

tb = memtest(data_w=dw, mem = MEM, addr_w = 9)
d = tb.elab(tgt, elab_all = True)
d = d[0]

c = tb.obj.ctx

d.run("hierarchy -check")
d.run("stat")
# d.run("write_rtlil mem8.il")
d.run("debug memory -nomap; debug opt")

 DEBUG CREATE wrapper module for transparent_r1w1 (EmulationModule 'top_transparent_r1w1') 
Creating process 'transparent_r1w1/mem_rw' with sensitivity (clk'rising,)
 Elaborating component transparent_r1w1_s1_s1_s9_s9_s16_s16_s512_0_16_9 
 Adding module with name `transparent_r1w1` 
 DEBUG: SKIP ARRAYTYPE `ram` : <class 'myirl.lists.SigArray'> 
 DEBUG: SKIP NON-SIGNAL ARGUMENT `MODE` : <class 'bool'> 
 DEBUG: SKIP NON-SIGNAL ARGUMENT `DWIDTH` : <class 'int'> 
 DEBUG: SKIP NON-SIGNAL ARGUMENT `AWIDTH` : <class 'int'> 
     DEBUG SLICE READ MEMORY to dout AWIDTH:9 DWIDTH:16
 [Component 'meminit/meminit'] blackbox not returning instances 
DEBUG ADD ROM 9:16
 DEBUG: Not adding `self` (type <class 'yosys.primitives._Builtins'>) to ports 
 PARAM PRIORITY --> 48 
 PARAM WORDS --> 512 
 PARAM ABITS --> 9 
 PARAM WIDTH --> 16 
 PARAM MEMID --> mem_ram_sig 
 [Component 'memrd/memrd'] blackbox not returning instances 
 DEBUG: Not adding `self` (type <class 'yosys.primitives._Builtins'>) to ports 

''


-- Running command `hierarchy -top \transparent_r1w1' --

1. Executing HIERARCHY pass (managing design hierarchy).

1.1. Analyzing design hierarchy..
Top module:  \transparent_r1w1

1.2. Analyzing design hierarchy..
Top module:  \transparent_r1w1
Removed 0 unused modules.

-- Running command `hierarchy -check' --

2. Executing HIERARCHY pass (managing design hierarchy).

2.1. Analyzing design hierarchy..
Top module:  \transparent_r1w1

2.2. Analyzing design hierarchy..
Top module:  \transparent_r1w1
Removed 0 unused modules.

-- Running command `stat' --

3. Printing statistics.

=== transparent_r1w1 ===

   Number of wires:                 17
   Number of wire bits:            182
   Number of public wires:           7
   Number of public wire bits:      68
   Number of memories:               1
   Number of memory bits:         8192
   Number of processes:              0
   Number of cells:                  7
     $and                            1
     $dff                          

## Testing hardware generation

Note: Post mapping does currently not support simulation via CXXRTL due to blackbox issues.

In [6]:
TECHMAP = '/usr/share/yosys'

# Read blackbox cells for awareness of DP16KD:
d.run("read_verilog -lib -specify %s/ecp5/cells_sim.v %s/ecp5/cells_bb.v" % (TECHMAP, TECHMAP))

# First try DP16KD mapping:
d.run("memory_bram -rules %s/ecp5/brams.txt" % TECHMAP)
d.run("techmap -map %s/ecp5/brams_map.v" % TECHMAP)

# Remaining (addr_w <= 8 bit) to LUT RAM:
d.run("memory_bram -rules %s/ecp5/lutrams.txt" % TECHMAP)
d.run("techmap -map %s/ecp5/lutrams_map.v" % TECHMAP)
d.run("opt_clean")
d.run("ls")

d.run("stat")

d.display_rtl(selection = MEM, fmt = 'dot')


-- Running command `read_verilog -lib -specify /usr/share/yosys/ecp5/cells_sim.v /usr/share/yosys/ecp5/cells_bb.v' --

6. Executing Verilog-2005 frontend: /usr/share/yosys/ecp5/cells_sim.v
Parsing Verilog input from `/usr/share/yosys/ecp5/cells_sim.v' to AST representation.
Generating RTLIL representation for module `\LUT4'.
Generating RTLIL representation for module `\$__ABC9_LUT5'.
Generating RTLIL representation for module `\$__ABC9_LUT6'.
Generating RTLIL representation for module `\$__ABC9_LUT7'.
Generating RTLIL representation for module `\L6MUX21'.
Generating RTLIL representation for module `\CCU2C'.
Generating RTLIL representation for module `\TRELLIS_RAM16X2'.
Generating RTLIL representation for module `\PFUMX'.
Generating RTLIL representation for module `\TRELLIS_DPR16X4'.
Generating RTLIL representation for module `\DPR16X4C'.
Generating RTLIL representation for module `\LUT2'.
Generating RTLIL representation for module `\TRELLIS_FF'.
Generating RTLIL representation for mod

In [7]:
from yosys import display
display.display_dot("memtest%d" % dw)